
#**📚 IMPORT LIBRARIES**

In [ ]:

!pip install -q torch torchvision timm mlflow pyngrok pytorch-lightning==2.4.0
!pip install -q "huggingface-hub<1.0" "transformers<4.44" torchmetrics==1.4.0

!pip install -q torch torchvision timm mlflow pyngrok pytorch-lightning==2.4.0
!pip install -q torchmetrics==1.3.1 "huggingface-hub<1.0" "transformers<4.44"

# CLEAN CACHE (optional but recommended)
import os, shutil
if os.path.exists("/root/.cache/huggingface/hub"):
    shutil.rmtree("/root/.cache/huggingface/hub")

import os
# os → file path operations, environment variables, folder handling

import shutil
# shutil → high-level file operations (copy, move, delete directories)

from pathlib import Path
# Path → clean, object-oriented file path handling

from PIL import Image
# PIL.Image → used for opening, transforming, and working with image files

import torch
# torch → main PyTorch deep learning engine (GPU tensors, autograd, nn modules)

import pytorch_lightning as pl
# pytorch_lightning → organizes training code into clean modules:
#                     LightningModule, Trainer, callbacks, checkpointing

from torch.utils.data import DataLoader
# DataLoader → loads dataset efficiently in mini-batches during training

from torchvision import datasets, transforms
# datasets → common datasets (ImageNet, CIFAR-10, MNIST, etc.)
# transforms → image preprocessing (resize, normalize, augmentations)

import timm
# timm → large collection of pretrained SOTA vision models (ViT, EfficientNet, ConvNeXt)

import mlflow
import mlflow.pytorch
# mlflow → log metrics, hyperparameters, plots, and model checkpoints
# mlflow.pytorch → saves/loads PyTorch models with MLflow

from pyngrok import ngrok
# ngrok → exposes local MLflow server to a public URL (useful in Colab)
#Example: http://1234.ngrok-free.app → MLflow UI accessible in browser


#**⚙️ Device Setup**

In [ ]:
# This line checks whether a GPU (CUDA) is available.
# If a GPU exists, training will run on the GPU for much faster performance.
# If no GPU is found (e.g., running on CPU-only machine), it will fall back to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print which device the model will use
print("Running on:", device)


Running on: cuda


#**🔧 USER CONFIGURATION**

In [ ]:
# =============================
# 🔧 USER CONFIGURATION
# (Edit these values according to your project)
# =============================

# 👉 Your Ngrok authentication token (required to create a public URL for MLflow UI)
# ⚠️ Keep this private. Anyone with this token can use your ngrok account.
NGROK_AUTH_TOKEN = "36CLviJ3eedsGnzgPHm7MM9g4De_4tDSxH3CgGEQCytxcXKgX"

# 👉 Directory where MLflow will store run logs, metrics, parameters, and model artifacts
MLRUNS_DIR = "/content/mlruns"

# 👉 Path to your dataset ZIP file (usually stored in Google Drive)
# ML pipeline will unzip and prepare the dataset from here.
DATA_ZIP = "/content/drive/MyDrive/new-plant-diseases-dataset.zip"

# 👉 Directory where the dataset should be extracted
EXTRACT_DIR = "/content/new-plant-diseases-dataset"

# 👉 Image size for training (input size for vision models like ViT, EfficientNet, ResNet)
IMG_SIZE = 224

# 👉 Number of images per batch during training
# Higher batch size → faster training but requires more GPU memory
BATCH_SIZE = 16

# 👉 Maximum number of epochs the model will train
# One epoch = one full pass over the entire dataset
MAX_EPOCHS = 2


#**✅ Ngrok Tunnel Reset**

In [ ]:
from pyngrok import ngrok
# pyngrok → a Python wrapper for ngrok, allowing you to create public URLs
#            for local services (like MLflow UI) directly from code.

ngrok.kill()
# This command closes ALL existing ngrok tunnels.
# Why?
# - Prevents "address already in use" errors
# - Ensures a clean state before creating a new ngrok tunnel
# - Useful in Google Colab where old tunnels may stay alive across runs



#**✅ MLflow + Ngrok Setup Function**

In [ ]:
def start_mlflow_ui(port: int = 5000, mlruns_dir: str = MLRUNS_DIR):

    # -----------------------------------------
    # Create the MLflow directory if not exists
    # -----------------------------------------
    os.makedirs(mlruns_dir, exist_ok=True)
    # MLflow stores metrics, params, and artifacts under this folder.


    # -----------------------------------------
    # Kill previous MLflow + ngrok processes
    # Ensures a clean start (very important in Colab)
    # -----------------------------------------
    try:
        get_ipython().system_raw("pkill -f mlflow")
        # Kills any process related to "mlflow ui" running in the background
    except:
        pass

    try:
        ngrok.kill()
        # Closes ALL running ngrok tunnels, preventing port conflicts
    except:
        pass


    # -----------------------------------------
    # Add your NGROK auth token (required for tunnels)
    # -----------------------------------------
    os.system(f"ngrok config add-authtoken {NGROK_AUTH_TOKEN}")
    # Registers your ngrok token so tunnels work under your account.


    # -----------------------------------------
    # FIX 1: Host header rewrite
    # Required because MLflow runs on a different internal URL.
    # This prevents 502 Bad Gateway errors.
    # -----------------------------------------
    os.system("mkdir -p /root/.ngrok2")
    os.system("echo 'host_header: rewrite' >> /root/.ngrok2/ngrok.yml")


    # -----------------------------------------
    # Start MLflow UI in the background
    # -----------------------------------------
    get_ipython().system_raw(
        f"nohup mlflow ui "
        f"--backend-store-uri file:{mlruns_dir} "  # Where mlruns are stored
        f"--port {port} "                          # Local MLflow port
        f"--host 0.0.0.0 "                         # Expose it for ngrok
        f"> /content/mlflow.log 2>&1 &"            # Log output to file
    )

    time.sleep(3)  # Wait for MLflow to start


    # -----------------------------------------
    # FIX 2: Use host_header="rewrite"
    # Required to make MLflow UI accessible via ngrok
    # -----------------------------------------
    public_url = ngrok.connect(port, "http", host_header="rewrite")


    # -----------------------------------------
    # Return public MLflow tracking URL
    # -----------------------------------------
    print("Your MLflow Tracking URL:", public_url)

    return public_url


#**✅ PlantDataModule — Lightning Data Module for Plant Images**

In [ ]:
# =====================================================
# 🌱 PlantDataModule for Plant Disease Classification
# =====================================================

import os  # File/folder operations (join paths, check existence)
import pytorch_lightning as pl  # Lightning framework for structured training
from torch.utils.data import DataLoader  # Wraps datasets into batches with shuffle & workers
from torchvision import datasets, transforms  # Dataset loading & image preprocessing

# ----------------------------------------
# LightningDataModule for Plant Disease Dataset
# ----------------------------------------
class PlantDataModule(pl.LightningDataModule):

    def __init__(self, data_dir, batch_size=32, img_size=224):
        """
        Initialize the DataModule.
        Args:
            data_dir (str): Root path to dataset
            batch_size (int): Images per batch
            img_size (int): Resize images to square size
        """
        super().__init__()  # Initialize LightningDataModule
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size

        # -------------------------------
        # Transformations for training images
        # -------------------------------
        self.train_tfms = transforms.Compose([
            transforms.Resize((img_size, img_size)),  # Resize all images
            transforms.RandomHorizontalFlip(),        # Data augmentation: random horizontal flip
            transforms.ToTensor()                     # Convert PIL Image → PyTorch Tensor
        ])

        # -------------------------------
        # Transformations for validation/test images
        # -------------------------------
        self.val_tfms = transforms.Compose([
            transforms.Resize((img_size, img_size)),  # Resize all images
            transforms.ToTensor()                     # Convert PIL Image → PyTorch Tensor
        ])

    # -------------------------------
    # Setup datasets (train/val/test)
    # -------------------------------
    def setup(self, stage=None):
        """
        Called by Lightning at the start of training/validation/testing.
        Loads datasets using ImageFolder.
        """
        train_dir = os.path.join(self.data_dir, "train")  # Path to training images
        val_dir = os.path.join(self.data_dir, "valid")    # Path to validation images
        test_dir = os.path.join(self.data_dir, "test")    # Path to test images

        # Safety check: training folder must exist
        if not os.path.exists(train_dir):
            raise FileNotFoundError(f"Expected train folder at {train_dir}")

        # Load datasets
        self.train_ds = datasets.ImageFolder(train_dir, transform=self.train_tfms)
        self.val_ds = datasets.ImageFolder(val_dir, transform=self.val_tfms) if os.path.exists(val_dir) else None
        self.test_ds = datasets.ImageFolder(test_dir, transform=self.val_tfms) if os.path.exists(test_dir) else None

    # -------------------------------
    # Training DataLoader
    # -------------------------------
    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,  # Shuffle each epoch for better generalization
            num_workers=2  # Parallel data loading
        )

    # -------------------------------
    # Validation DataLoader
    # -------------------------------
    def val_dataloader(self):
        return DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            num_workers=2
        ) if self.val_ds else None  # Returns None if no validation dataset

    # -------------------------------
    # Test DataLoader
    # -------------------------------
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=2
        ) if self.test_ds else None  # Returns None if no test dataset


#**🌱 PlantDiseaseModel — PyTorch Lightning Module**

In [ ]:
# =====================================================
# 🌱 PlantDiseaseModel — PyTorch Lightning Module
# =====================================================

import pytorch_lightning as pl  # Lightning framework for structured training
import torch                     # Core PyTorch library
import timm                      # Library with SOTA pretrained vision models

# -------------------------------
# Learning rate
# -------------------------------
LR = 1e-4  # Default learning rate for optimizer

# -------------------------------
# LightningModule for Plant Disease Classification
# -------------------------------
class PlantDiseaseModel(pl.LightningModule):
    def __init__(self, model_name="efficientnet_b0", lr=LR, num_classes=2):
        """
        Initialize the model.
        Args:
            model_name (str): Pretrained model from timm
            lr (float): Learning rate
            num_classes (int): Number of output classes
        """
        super().__init__()

        # Save hyperparameters for checkpointing and logging
        self.save_hyperparameters()

        # -------------------------------
        # Backbone model (pretrained)
        # -------------------------------
        self.backbone = timm.create_model(
            model_name,       # e.g., efficientnet_b0
            pretrained=True,  # Load pretrained ImageNet weights
            num_classes=num_classes  # Final layer output classes
        )

        # -------------------------------
        # Loss function
        # -------------------------------
        self.loss_fn = torch.nn.CrossEntropyLoss()
        # CrossEntropyLoss is standard for multi-class classification

    # -------------------------------
    # Forward pass
    # -------------------------------
    def forward(self, x):
        """
        Forward pass through the model.
        Args:
            x: Input tensor of shape (B, C, H, W)
        Returns:
            logits: Raw class scores
        """
        return self.backbone(x)

    # -------------------------------
    # Training step
    # -------------------------------
    def training_step(self, batch, batch_idx):
        """
        One training step.
        Args:
            batch: Tuple (images, labels)
            batch_idx: Index of batch (not used here)
        Returns:
            loss: Used for backpropagation
        """
        x, y = batch
        logits = self(x)                       # Forward pass
        loss = self.loss_fn(logits, y)         # Compute loss
        acc = (logits.argmax(dim=1) == y).float().mean()  # Compute batch accuracy

        # Log metrics to Lightning progress bar & logger
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        return loss

    # -------------------------------
    # Validation step
    # -------------------------------
    def validation_step(self, batch, batch_idx):
        """
        One validation step.
        Args:
            batch: Tuple (images, labels)
            batch_idx: Index of batch
        """
        x, y = batch
        logits = self(x)                       # Forward pass
        loss = self.loss_fn(logits, y)         # Compute validation loss
        acc = (logits.argmax(dim=1) == y).float().mean()  # Compute accuracy

        # Log metrics (shown in progress bar)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    # -------------------------------
    # Optimizer configuration
    # -------------------------------
    def configure_optimizers(self):
        """
        Define optimizer for training.
        Returns:
            optimizer: Adam optimizer with learning rate self.hparams.lr
        """
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


#**💾 Mount Google Drive + Prepare Dataset**

In [ ]:
# Mount Drive (optional) + prepare dataset
# -----------------------------
from google.colab import drive
import os, shutil
from pathlib import Path

print("Mounting Drive (if not already mounted)...")
drive.mount('/content/drive', force_remount=False)

# If user provided a zip on Drive, extract it to EXTRACT_DIR
if os.path.exists(DATA_ZIP) and not os.path.exists(EXTRACT_DIR):
    print(f"Extracting {DATA_ZIP} to {EXTRACT_DIR} ...")
    shutil.unpack_archive(DATA_ZIP, EXTRACT_DIR)

# detect dataset dir
if os.path.exists(EXTRACT_DIR):
    entries = [p for p in Path(EXTRACT_DIR).iterdir() if p.is_dir()]
    DATA_DIR = str(entries[0]) if len(entries) == 1 else EXTRACT_DIR
else:
    # fallback - ask user to set DATA_DIR variable
    DATA_DIR = EXTRACT_DIR

print("DATA_DIR =", DATA_DIR)

# -----------------------------
# Prepare data module and model (will raise error if DATA_DIR not present)
# -----------------------------
print("Preparing data module and model (this will error if your dataset path is wrong)...")
dm = PlantDataModule(DATA_DIR)

try:
    dm.setup()
except Exception as e:
    raise RuntimeError(f"Data setup failed: {e}")

num_classes = len(dm.train_ds.classes)
print("Classes:", dm.train_ds.classes)

model = PlantDiseaseModel(num_classes=num_classes)

Mounting Drive (if not already mounted)...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DATA_DIR = /content/new-plant-diseases-dataset/new-plant-diseases-dataset
Preparing data module and model (this will error if your dataset path is wrong)...
Classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'P

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

# **⚡ SPEED BOOST SETTINGS + MLflow + Ngrok**

In [ ]:
def start_mlflow_ui(port: int = 5000, mlruns_dir: str = MLRUNS_DIR):

    # -----------------------------------------
    # Create the MLflow directory if not exists
    # -----------------------------------------
    os.makedirs(mlruns_dir, exist_ok=True)

    # -----------------------------------------
    # Kill previous MLflow + ngrok processes (More Aggressive Cleanup)
    # -----------------------------------------
    try:
        # Stop background jobs and aggressively kill any processes that might be running MLflow or ngrok
        get_ipython().system_raw("pkill -9 -f 'mlflow' || true")
        get_ipython().system_raw("pkill -9 -f 'ngrok' || true")
    except:
        pass

    try:
        ngrok.kill()
    except:
        pass

    time.sleep(2)  # Wait for processes to fully shut down (Crucial!)


    # -----------------------------------------
    # Add your NGROK auth token (required for tunnels)
    # -----------------------------------------
    os.system(f"ngrok config add-authtoken {NGROK_AUTH_TOKEN}")

    # -----------------------------------------
    # FIX 1: Host header rewrite
    # -----------------------------------------
    os.system("mkdir -p /root/.ngrok2")
    os.system("echo 'host_header: rewrite' >> /root/.ngrok2/ngrok.yml")


    # -----------------------------------------
    # Start MLflow UI in the background
    # -----------------------------------------
    get_ipython().system_raw(
        f"nohup mlflow ui "
        f"--backend-store-uri file:{mlruns_dir} "
        f"--port {port} "
        f"--host 0.0.0.0 "
        f"> /content/mlflow.log 2>&1 &"
    )

    time.sleep(3)  # Wait for MLflow to start


    # -----------------------------------------
    # FIX 2: Use host_header="rewrite"
    # -----------------------------------------
    public_url = ngrok.connect(port, "http", host_header="rewrite")


    # -----------------------------------------
    # Return public MLflow tracking URL
    # -----------------------------------------
    print("Your MLflow Tracking URL:", public_url)

    return public_url

# **🚀 Lightning Trainer + MLflow Logging + Model Save**

In [ ]:
import os
import time
import torch
import pytorch_lightning as pl
import mlflow
import pytorch_lightning.loggers as pl_loggers # <--- ADDED IMPORT

print("Starting MLflow UI + ngrok...")
public_url = start_mlflow_ui(port=5000, mlruns_dir=MLRUNS_DIR)
print("🔗 MLflow URL:", public_url)

# --- Initialize MLflow Logger ---
# 1. Initialize the MLflow logger, linking it to your local tracking directory
mlflow_logger = pl_loggers.MLFlowLogger(
    experiment_name="plant_disease_experiment",
    tracking_uri=f"file:{MLRUNS_DIR}"
)

# ----------------------------------------
# Initialize PyTorch Lightning Trainer (UPDATED)
# ----------------------------------------
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,  # Total number of training epochs
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    log_every_n_steps=10,
    # 👇 EXPLICITLY PASS THE LOGGER TO THE TRAINER
    logger=mlflow_logger
)
# - Lightning handles training loop, validation, and logging automatically

# ----------------------------------------
# Start MLflow experiment run
# ----------------------------------------
# Note: When using pl.loggers.MLFlowLogger, mlflow.start_run() is technically
# optional, but we keep it here to ensure parameters are logged easily.
with mlflow.start_run(run_name="efficientnet_training"):
    # -------------------------------
    # Log hyperparameters
    # -------------------------------
    mlflow.log_params({
        "img_size": IMG_SIZE,
        "batch_size": BATCH_SIZE,
        "epochs": MAX_EPOCHS,
        "model_name": model.hparams.model_name, # Log the EfficientNet name
        "learning_rate": model.hparams.lr      # Log the learning rate
    })

    # -------------------------------
    # Fit model
    # -------------------------------
    trainer.fit(model, dm)
    # - Trainer uses the 'mlflow_logger' to automatically record train/val metrics

    # -------------------------------
    # Optional validation
    # -----------------------------

# SAVE MODEL HERE
MODEL_PATH = "model.pth"
# Use the model's state_dict() for saving the raw weights
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved at:", MODEL_PATH)

num_classes = len(dm.train_ds.classes)
num_classes

Starting MLflow UI + ngrok...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Your MLflow Tracking URL: NgrokTunnel: "https://36faaa900b34.ngrok-free.app" -> "http://localhost:5000"
🔗 MLflow URL: NgrokTunnel: "https://36faaa900b34.ngrok-free.app" -> "http://localhost:5000"


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | EfficientNet     | 4.1 M  | train
1 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.225    Total estimated model params size (MB)
338       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
dm.train_ds.classes

In [ ]:
# SAVE MODEL HERE
MODEL_PATH = "model.pth"
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved at:", MODEL_PATH)

In [ ]:
num_classes = len(dm.train_ds.classes)
num_classes

In [ ]:
for idx, cls in enumerate(dm.train_ds.classes):
    print(idx, cls)
